In [4]:
import sympy as sp
import numpy as np
import time 

N = 3
g = sp.Symbol("g", real = True)

## Creating X coordinates (using sp.Matrix for the 4th stack)

In [5]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,4):
    for j in np.arange(1,10):
        X[str(i)+str(j)] = sp.Symbol("X^{"+str(i)+"}_"+str(j), real = True)
            
for i in np.arange(1,10):
    #initialise a matrix of correct shape
    X_init = []
    for j in np.arange(1,N+1):
        row = []
        for k in np.arange(1,N+1):
            row.append(sp.Symbol("a_{"+str(j)+str(k)+"}"))
        X_init.append(row)
    X["4"+str(i)] = sp.Matrix(X_init)
    
    for j in np.arange(N):
        for k in np.arange(N):
            if j == k:
                X["4"+str(i)][j,k] = sp.Symbol("x^{4"+str(i)+"}_{"+str(j+1)+str(k+1)+"}",real = True)
            elif j > k:
                continue
            else:
                x_ij = sp.Symbol("x^{4"+str(i)+"}_{"+str(j+1)+str(k+1)+"}",real = True)
                y_ij = sp.Symbol("y^{4"+str(i)+"}_{"+str(j+1)+str(k+1)+"}",real = True)
                X["4"+str(i)][j,k] = x_ij + sp.I*y_ij
                X["4"+str(i)][k,j] = x_ij - sp.I*y_ij

## Creating X coordinates (using sp.MatrixSymbol for the 4th stack)

In [3]:
# Creating a dictionary to store the X coordinates
X = {}
for i in np.arange(1,5):
    for j in np.arange(1,10):
        if i != 4:
            X[str(i)+str(j)] = sp.Symbol("X^{"+str(i)+"}_"+str(j), real = True)
        else:
            X[str(i)+str(j)] = sp.MatrixSymbol("X^{"+str(i)+"}_"+str(j),N,N)

## Creating the $\phi$'s and $\phi^{\dagger}$'s and $\phi^{(ij)}$

In [6]:
phi = {}
phi_ = {}
phi_dag = {}
for i in np.arange(1,5):
    phi[str(i)+"1"] = 1/sp.sqrt(2) * (X[str(i)+"4"] + sp.I * X[str(i)+"5"])
    phi[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] + sp.I * X[str(i)+"7"])
    phi[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] + sp.I * X[str(i)+"9"])
    
    phi_dag[str(i)+"1"] = 1/sp.sqrt(2) * ( X[str(i)+"4"] - sp.I * X[str(i)+"5"])
    phi_dag[str(i)+"2"] = 1/sp.sqrt(2) * (X[str(i)+"6"] - sp.I * X[str(i)+"7"])
    phi_dag[str(i)+"3"] = 1/sp.sqrt(2) * (X[str(i)+"8"] - sp.I * X[str(i)+"9"])
    
phi_["12"] = phi["13"] - phi["23"]
phi_["23"] = phi["21"] - phi["31"]
phi_["31"] = phi["32"] - phi["12"]

## Creating Z coordinates

In [7]:
# Building the Z-dictionary
Z = {}
for i in np.arange(1,4):
    #Scalars here
    for j in np.arange(1,4):
        if i == j: 
            continue
        else:
            P = sp.Symbol("P^{"+str(i) + str(j) +"}", real = True)
            Q = sp.Symbol("Q^{"+str(i) + str(j) +"}", real = True)
            Z[str(i) + str(j)] = (P + sp.I * Q)
    
    #Vectors here
    store_current_4i = []
    store_current_i4 = []
    for j in np.arange(1,N+1):
        P4i = sp.Symbol("P^{"+str(4)+str(i)+"}_"+str(j), real = True)
        Q4i = sp.Symbol("Q^{"+str(4)+str(i)+"}_"+str(j), real = True)
        Pi4 = sp.Symbol("P^{"+str(i)+str(4)+"}_"+str(j), real = True)
        Qi4 = sp.Symbol("Q^{"+str(i)+str(4)+"}_"+str(j), real = True)
        store_current_4i.append([P4i + sp.I * Q4i])
        store_current_i4.append([Pi4 + sp.I * Qi4])
    Z["4"+str(i)] = sp.Matrix(store_current_4i)
    Z[str(i) + "4"] = sp.Matrix(store_current_i4).transpose()

## Creating the F variables

In [11]:
F = {}

#Scalars
for i in np.arange(1,4):
    for j in np.arange(1,5):
        if i == j:
            continue
        if j == 4:
            F[str(i)+str(j)] = (Z[str(i)+str(j)] * Z[str(j)+str(i)])[0] + N*sp.Symbol("c^{"+str(i)+str(j)+"}")/g**2
        else:
            F[str(i)+str(j)] = Z[str(i)+str(j)] * Z[str(j)+str(i)] + sp.Symbol("c^{"+str(i)+str(j)+"}")/g**2 

F["41"] = Z["41"]*Z["14"] + (sp.Symbol("c^{41}")/g**2)*sp.eye(N) +  phi["42"]*phi["43"] - phi["43"]*phi["42"]
F["42"] = Z["42"]*Z["24"] + (sp.Symbol("c^{42}")/g**2)*sp.eye(N) +  phi["43"]*phi["41"] - phi["41"]*phi["43"]
F["43"] = Z["43"]*Z["34"] + (sp.Symbol("c^{43}")/g**2)*sp.eye(N) -  phi["42"]*phi["41"] + phi["41"]*phi["42"]

## Writing in the G's

In [21]:
G = {}

G["21"] = phi_["12"]*Z["21"] + Z["23"]*Z["31"] + (Z["24"]*Z["41"])[0]
G["12"] = phi_["12"]*Z["12"] + Z["13"]*Z["32"] + (Z["14"]*Z["42"])[0]

G["31"] = phi_["31"]*Z["31"] + Z["32"]*Z["21"] - (Z["34"]*Z["41"])[0]
G["13"] = phi_["31"]*Z["13"] + Z["12"]*Z["23"] + (Z["14"]*Z["43"])[0]

G["32"] = phi_["23"]*Z["32"] + Z["31"]*Z["12"] + (Z["34"]*Z["42"])[0]
G["23"] = phi_["23"]*Z["23"] + Z["21"]*Z["13"] + (Z["24"]*Z["43"])[0]

G["41"] = -phi["41"]*Z["41"] + Z["42"]*Z["21"] + Z["43"]*Z["31"]
G["14"] = -Z["14"]*phi["41"] + Z["12"]*Z["24"] - Z["13"]*Z["34"]

G["42"] = -phi["42"]*Z["42"] + Z["43"]*Z["32"] + Z["41"]*Z["12"]
G["24"] = -Z["24"]*phi["42"] + Z["21"]*Z["14"] + Z["23"]*Z["34"]

G["43"] = -phi["43"]*Z["43"] - Z["41"]*Z["13"] + Z["42"]*Z["23"]
G["34"] = -Z["34"]*phi["43"] + Z["31"]*Z["14"] + Z["32"]*Z["24"]

scalar_G = [G["12"],G["21"],G["13"],G["31"],G["23"],G["32"]]

## Creating $V_F$

In [41]:
start = time.time()
Fsum1 = sp.Symbol("",real = True)
Fsum2 = sp.Symbol("",real = True)
Fsum3 = sp.Symbol("",real = True)
Gsum1 = sp.Symbol("",real = True)
Gsum2 = sp.Symbol("",real = True)
Gsum3 = sp.Symbol("",real = True)

#Term 1
for i in np.arange(1,4):
    for j in np.arange(1,4):
        if i == j:
            continue
        else:
            Fsum1+= sp.re(F[str(i)+str(j)])**2 + sp.im(F[str(i)+str(j)])**2
#Term 2
for i in np.arange(1,4):
    Fsum2 += sp.re(F[str(i)+"4"])**2 + sp.im(F[str(i)+"4"])**2
    
#Term 3
for i in np.arange(1,4):
    for j in np.arange(N):
        for k in np.arange(N):
            Fsum3 += sp.re(F["4"+str(i)][j,k])**2 + sp.im(F["4"+str(i)][j,k])**2
            
#Term 4
for i in np.arange(6):
    Gsum1 += (sp.re(scalar_G[i]))**2 + (sp.im(scalar_G[i]))**2
    
#Term 5
for i in np.arange(1,4):
    for j in np.arange(N):
        Gsum2 += (sp.re(G[str(i)+"4"][j]))**2 + (sp.im(G[str(i)+"4"][j]))**2

#Term 6
for i in np.arange(1,4):
    for j in np.arange(N):
        Gsum3 += (sp.re(G[str(i)+"4"][j]))**2 + (sp.im(G[str(i)+"4"][j]))**2
        
V_F = Fsum1 + Fsum2 + Fsum3 + Gsum1 + Gsum2 + Gsum3

print(time.time() - start)

0.059001922607421875


In [47]:
(V_F.diff(sp.re(Z["12"])))

2*P^{21}*(P^{12}*P^{21} - Q^{12}*Q^{21} + re(c^{12})/g**2) + 2*P^{21}*(P^{12}*P^{21} - Q^{12}*Q^{21} + re(c^{21})/g**2) + 2*P^{23}*(P^{12}*P^{23} - sqrt(2)*P^{13}*X^{1}_6/2 + sqrt(2)*P^{13}*X^{3}_6/2 + P^{14}_1*P^{43}_1 + P^{14}_2*P^{43}_2 + P^{14}_3*P^{43}_3 - Q^{12}*Q^{23} + sqrt(2)*Q^{13}*X^{1}_7/2 - sqrt(2)*Q^{13}*X^{3}_7/2 - Q^{14}_1*Q^{43}_1 - Q^{14}_2*Q^{43}_2 - Q^{14}_3*Q^{43}_3) + 4*P^{24}_1*(P^{12}*P^{24}_1 - P^{13}*P^{34}_1 - Q^{12}*Q^{24}_1 + Q^{13}*Q^{34}_1 + sqrt(2)*(-P^{14}_1*x^{44}_{11} + Q^{14}_1*x^{45}_{11})/2 + sqrt(2)*(-P^{14}_2*x^{44}_{12} - P^{14}_2*y^{45}_{12} + Q^{14}_2*x^{45}_{12} - Q^{14}_2*y^{44}_{12})/2 + sqrt(2)*(-P^{14}_3*x^{44}_{13} - P^{14}_3*y^{45}_{13} + Q^{14}_3*x^{45}_{13} - Q^{14}_3*y^{44}_{13})/2) + 4*P^{24}_2*(P^{12}*P^{24}_2 - P^{13}*P^{34}_2 - Q^{12}*Q^{24}_2 + Q^{13}*Q^{34}_2 + sqrt(2)*(-P^{14}_2*x^{44}_{22} + Q^{14}_2*x^{45}_{22})/2 + sqrt(2)*(-P^{14}_1*x^{44}_{12} + P^{14}_1*y^{45}_{12} + Q^{14}_1*x^{45}_{12} + Q^{14}_1*y^{44}_{12})/2 + sqrt(